In [ ]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import pytz
import calendar

In [ ]:
df = pd.read_csv('forest-of-sivakar.csv')
print(df.tail())


# Preprocessing

In [ ]:
# Drop unnecessary columns
columns_to_drop = ['Note', 'Tree Type', 'Is Success']
df = df.drop(columns=columns_to_drop)


def parse_timestamp(ts):
    dt = pd.to_datetime(ts, format='%a %b %d %H:%M:%S GMT%z %Y')
    return dt.tz_convert(pytz.timezone('Asia/Kolkata'))

def split_at_midnight(row):
    start = row['Start Time']
    end = row['End Time']

    local_tz = start.tz
    midnight = (start + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)

    if start.date() != end.date():
        first_half = pd.Series({
            'Start Time': start,
            'End Time': (midnight - timedelta(seconds=1)).tz_convert(local_tz)
        })

        second_half = pd.Series({
            'Start Time': midnight.tz_convert(local_tz),
            'End Time': end
        })

        return [first_half, second_half]

    return [row]

# Parse timestamps
df['Start Time'] = df['Start Time'].apply(parse_timestamp)
df['End Time'] = df['End Time'].apply(parse_timestamp)

# Apply the split_at_midnight function and explode the result
split_rows = []
for _, row in df.iterrows():
    split_rows.extend(split_at_midnight(row))
df = pd.DataFrame(split_rows)
df.reset_index()

# Display the first few rows of the processed dataframe
print(df.head(10))

# Optionally, save the processed data to a new CSV file
# df.to_csv('processed_data.csv', index=False)

In [ ]:
df.info()

In [ ]:
# Calculate duration for each entry
df['Duration'] = df['End Time'] - df['Start Time']

# Analytics

## Total Time Each Year

In [ ]:

# Extract year from Start Time
df['Year'] = df['Start Time'].dt.year

# Group by year and sum the durations, then convert to hours
yearly_totals = df.groupby('Year')['Duration'].sum().dt.total_seconds() / 3600

# Round the hours to 2 decimal places
yearly_totals = yearly_totals.round(2)

# The result is now a Series with Year as index and Total Hours as values
print(yearly_totals)

In [ ]:
yearly_totals.plot.bar()

## Total Time Each Month

In [ ]:
# Extract year and month from Start Time
df['Year-Month'] = df['Start Time'].dt.to_period('M')

# Group by year-month and sum the durations, then convert to hours
monthly_totals = df.groupby('Year-Month')['Duration'].sum().dt.total_seconds() / 3600

# Round the hours to 2 decimal places
monthly_totals = monthly_totals.round(2)

# The result is now a Series with Year-Month as index and Total Hours as values
print(monthly_totals)

In [ ]:
# Set up the plot
plt.figure(figsize=(20, 6))  # Adjust the width (20) and height (6) as needed

# Create a color map for years
years = monthly_totals.index.year.unique()
color_map = plt.colormaps['tab10']  # You can change 'tab10' to other color maps
colors = {year: color_map(i/len(years)) for i, year in enumerate(years)}

# Create lists to hold x-positions and colors for each bar
x_positions = []
bar_colors = []
labels = []

# Prepare data for plotting
for i, (date, value) in enumerate(monthly_totals.items()):
    month = date.month
    year = date.year
    x_positions.append(month + (year - min(years)) * 12)
    bar_colors.append(colors[year])
    labels.append(f"{calendar.month_abbr[month]}\n{year}")

# Plot bars
plt.bar(x_positions, monthly_totals.values, color=bar_colors, width=0.8)

# Customize the plot
plt.title('Total Hours per Month', fontsize=16)
plt.xlabel('Month-Year', fontsize=12)
plt.ylabel('Total Hours', fontsize=12)
plt.xticks(x_positions, labels, rotation=45, ha='right')

# Add legend
for year in years:
    plt.bar(0, 0, color=colors[year], label=str(year))
plt.legend(title='Year', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()

# Show the plot
plt.show()

## Distribution of Daily Totals

In [ ]:
# Extract date from Start Time
df['Date'] = df['Start Time'].dt.date

# Group by date and sum the durations, then convert to minutes
daily_totals = df.groupby('Date')['Duration'].sum().dt.total_seconds() / 3600

# Round the minutes to 2 decimal places
daily_totals = daily_totals.round(2)

# The result is now a Series with Date as index and Total Minutes as values
print(daily_totals.tail())

In [ ]:
plt.figure(figsize=(12, 6))

plt.hist(daily_totals, bins=range(0, int(daily_totals.max()) + 2, 1),
         edgecolor='black', align='left')

plt.title('Distribution of Daily Total Hours', fontsize=16)
plt.xlabel('Hours', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

plt.xticks(range(0, int(daily_totals.max()) + 2, 1))

plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Analytics by Tags

In [ ]:
tags_counts = df['Tag'].value_counts()
tags_counts.head(10)

In [ ]:
tags_counts.plot.bar()